In [1]:
from flask import Flask, render_template, request

In [2]:
#!pip install flask-wtf

In [3]:
from flask_wtf import FlaskForm
from wtforms import IntegerField 

In [4]:
import numpy as np
import pandas as pd
import joblib

In [5]:
from sklearn.naive_bayes import GaussianNB

In [6]:
app = Flask(__name__, template_folder='template')


app.config['SECRET_KEY'] = 'mlpsfselfhhfowcp'

class FlaskAlarm(FlaskForm):
    Ambient_Temperature = IntegerField("Ambient Temperature")
    Calibration = IntegerField("Calibration")
    Unwanted_substance_deposition = IntegerField("Unwanted substance deposition")
    Humidity = IntegerField("Humidity")
    H2S_Content = IntegerField("H2S Content")
    detected_by = IntegerField("detected by")
    
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/train')
def train():
    df_train = pd.read_excel('False Alarm Cases.xlsx')
    df_train = df_train.iloc[:,1:8]
    x = df_train.iloc[:,0:6]
    y = df_train['Spuriosity Index(0/1)']
    model = GaussianNB()
    model.fit(x,y)
    filename = 'ml_model.pkl'
    joblib.dump(model,filename)
    return render_template('train.html', name=filename)

@app.route('/test', methods=['GET','POST'])
def test():
    form = FlaskAlarm()
    if request.method == "POST":
        filename = 'ml_model.pkl'
        clf = joblib.load(filename) # load ml model from joblib
        a = form.data ['Ambient_Temperature']
        b = form.data ['Calibration']
        c = form.data ['Unwanted_substance_deposition']
        d = form.data ['Humidity']
        e = form.data ['H2S_Content']
        f = form.data ['detected_by']
        cols = [a,b,c,d,e,f]
        array = np.array(cols)
        array = array.reshape(1,6)
        df_test = pd.DataFrame(array, columns=['Ambient Temperature','Calibration','Unwanted substance deposition','Humidity','H2S Content','detected by'])
        
        ypred = clf.predict(df_test)
        if ypred == 1:
            result = 'Danger ges'
        else:
            result = 'Not a danger ges'
        return "Prediction is %s"%(result)
        
    else:
        return render_template('test.html', form = form)

@app.route('/report')
def report():
    return render_template('report.html')

@app.route('/help')
def help():
    return render_template('help.html')


if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2023 18:48:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 18:48:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Feb/2023 18:48:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 18:48:53] "GET /report HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 18:48:54] "GET /help HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 18:48:58] "GET /train HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 18:49:01] "GET /test HTTP/1.1" 200 -
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Ambient Temperature
- Calibration
- H2S Content
- Humidity
- Unwanted substance deposition
- ...
Feature names seen at fit time, yet now missing:
- Ambient Temperature( deg C)
- Calibration(days)
- H2S Content(ppm)
- Humidity(%)
- Unwanted substance deposition(0